# Waveform Inversion for Subsurface Imaging - Kaggle Competition

### 1. Imports and Setup
 In this section, I import all the required libraries.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# 2. Load and Inspect Data
I want to understand the shapes and patterns in the training data

In [ ]:
train_data_dir = Path("/kaggle/input/waveform-inversion/train_samples/FlatVel_A")

# Load one sample to inspect shapes
velocity = np.load(train_data_dir / "model/model2.npy")
data = np.load(train_data_dir / "data/data2.npy")
print("Velocity map shape:", velocity.shape)
print("Seismic data shape:", data.shape)

# 3. Define Custom Dataset Class
I create a PyTorch dataset that loads seismic data and its corresponding velocity maps


In [ ]:
class FWI_Dataset(Dataset):
    def __init__(self, data_paths, model_paths):
        self.data_paths = data_paths
        self.model_paths = model_paths

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        x = np.load(self.data_paths[idx])  # shape: (samples, sources, time, receivers)
        y = np.load(self.model_paths[idx])  # shape: (samples, height, width)

        # Normalize seismic data sample-wise
        x = (x - x.mean()) / (x.std() + 1e-6)

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# 4. Build the Model
I'll use a small U-Net architecture to map seismic data to velocity maps


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder1 = DoubleConv(1, 32)
        self.pool1 = nn.MaxPool2d(2)
        self.encoder2 = DoubleConv(32, 64)

        self.up = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.decoder = DoubleConv(64, 32)
        self.output_layer = nn.Conv2d(32, 1, kernel_size=1)

    def forward(self, x):
        x1 = self.encoder1(x)
        x2 = self.encoder2(self.pool1(x1))
        x = self.up(x2)
        x = torch.cat([x1, x], dim=1)
        x = self.decoder(x)
        return self.output_layer(x)

# 5. Train the Model
Here I’ll define a simple training loop

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for x, y in loader:
        x = x.unsqueeze(1).to(device)  # Add channel dim
        y = y.unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# 6. Inference and Submission
Predict and reshape into format

In [ ]:
def predict_and_format(model, test_loader, sample_submission):
    model.eval()
    predictions = []
    with torch.no_grad():
        for x in test_loader:
            x = x.unsqueeze(1).to(device)
            preds = model(x).squeeze(1).cpu().numpy()
            predictions.extend(preds)

    # Only keep odd-indexed columns
    output = []
    for i, pred in enumerate(predictions):
        for y in range(pred.shape[0]):
            row = pred[y, ::2]  # odd-indexed columns
            output.append([f"testoid_y_{y}"] + row.tolist())

    columns = ["oid_ypos"] + sample_submission.columns[1:].tolist()
    df = pd.DataFrame(output, columns=columns)
    return df


# 7. Visualizations
I use a quick look at one seismic example and its corresponding velocity map


In [ ]:
def visualize_sample(data_path, model_path):
    seismic = np.load(data_path)[0]  # (sources, time, receivers)
    velocity = np.load(model_path)[0]  # (height, width)

    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].imshow(seismic.mean(axis=0), cmap="seismic", aspect='auto')
    axs[0].set_title("Seismic Data (mean over sources)")
    axs[1].imshow(velocity, cmap="viridis")
    axs[1].set_title("Velocity Map")
    plt.show()

In [ ]:
# Pair of files to visualize
sample_data_path = "/kaggle/input/waveform-inversion/train_samples/FlatVel_A/data/data2.npy"
sample_model_path = "/kaggle/input/waveform-inversion/train_samples/FlatVel_A/model/model2.npy"

visualize_sample(sample_data_path, sample_model_path)